In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np

import requests
import time
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import collections
import timeit
import re
from datetime import  datetime
import pymongo
import json

In [21]:
animals=Path("Resources/Health_AnimalBites.csv")
df_animals=pd.read_csv(animals)
df_animals.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc
0,1985-05-05 00:00:00,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
1,1986-02-12 00:00:00,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
2,1987-05-07 00:00:00,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
3,1988-10-02 00:00:00,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
4,1989-08-29 00:00:00,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaN,UNKNOWN,NaN,NaN,UNKNOWN


In [29]:
df_animals.dropna(subset = ['bite_date','victim_zip','SpeciesIDDesc','BreedIDDesc'],inplace=True)
df_animals['bite_year']=df_animals['bite_date'].apply(lambda x: int(x[:4]) if pd.notnull(x) else 0)
df_animals=df_animals[df_animals['bite_year']>=2013].reset_index(drop=True)
#df_animals.drop(['bite_date'],axis=1,inplace=True)
df_animals.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,year,bite_year
0,2014-03-24 00:00:00,DOG,PIT BULL,NaN,WHITE,NaN,NaN,40205,NaN,BODY,NaN,NaN,2017-03-24 00:00:00,NaN,NEGATIVE,2014,2014
1,2014-05-22 00:00:00,DOG,PIT BULL,NaN,GRAY/WHITE,NaN,NaN,40211,NaN,BODY,NaN,NaN,2014-05-22 00:00:00,NaN,NaN,2014,2014
2,2014-12-02 00:00:00,DOG,BEAGLE,FEMALE,TRI,NaN,2014-04-01 00:00:00,40208,NaN,HEAD,NaN,NaN,2014-12-05 00:00:00,NaN,NEGATIVE,2014,2014
3,2015-04-30 00:00:00,DOG,AAUST. TERR.,FEMALE,BROWN,NaN,NaN,40220,NO,BODY,NaN,KILLED,2015-04-30 00:00:00,NaN,NEGATIVE,2015,2015
4,2015-05-11 00:00:00,DOG,LABRADOR RETRIV,FEMALE,GOLDEN,NaN,NaN,40047,NO,UNKNOWN,NaN,KILLED,2015-05-18 00:00:00,NaN,NEGATIVE,2015,2015


In [35]:
columns=['bite_year','victim_zip','SpeciesIDDesc','BreedIDDesc','color','GenderIDDesc'
         ,'vaccination_yrs','vaccination_date'
         ,'WhereBittenIDDesc']
df_animals=df_animals[columns]
#print(df_animals.info())
df_animals.head()

,bite_year,victim_zip,SpeciesIDDesc,BreedIDDesc,color,GenderIDDesc,vaccination_yrs,vaccination_date,WhereBittenIDDesc
0,2014,40205,DOG,PIT BULL,WHITE,NaN,NaN,NaN,BODY
1,2014,40211,DOG,PIT BULL,GRAY/WHITE,NaN,NaN,NaN,BODY
2,2014,40208,DOG,BEAGLE,TRI,FEMALE,NaN,2014-04-01 00:00:00,HEAD
3,2015,40220,DOG,AAUST. TERR.,BROWN,FEMALE,NaN,NaN,BODY
4,2015,40047,DOG,LABRADOR RETRIV,GOLDEN,FEMALE,NaN,NaN,UNKNOWN


In [36]:
# Use PyMongo to establish Mongo connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [37]:
# Define database and collection
db = client.AnimalBites
bites = db.Bites

In [38]:
#Convert dataframe to List of Json object
records = json.loads(df_animals.T.to_json()).values()

In [39]:
try:
    bites.drop()
    bites.insert_many(records)
except Exception as e:
    print(e)   

In [42]:
len(df_animals['BreedIDDesc'].unique())

95